In [8]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.cluster import KMeans
import pickle

from scipy.spatial.distance import cdist

In [9]:
def corvert_to_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def SIFT(image):
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(image, None)
    return  descriptors

In [10]:
def kmeans_bow(all_descriptors, num_clusters):

    bow_dict = []
    kmeans = KMeans(n_clusters=num_clusters, verbose = 1).fit(all_descriptors)
    bow_dict = kmeans.cluster_centers_
    
    return bow_dict

In [11]:
path = 'D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix'

features = []
image_features = []
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        image = cv2.imread(os.path.join(dirname, filename))
        gray = corvert_to_gray(image)
        sift = SIFT(gray)
        image_features.append(sift)
        for i in sift:
            features.append(i)

In [12]:
num_clusters = 50
file_path = os.path.join(os.getcwd(), 'bow_dictionary.pkl')
if not os.path.isfile(file_path):
    BoW = kmeans_bow(features, num_clusters)
    with open(file_path, 'wb') as file:
        pickle.dump(BoW, file)
else:
    with open(file_path, 'rb') as file:
            BoW = pickle.load(file)

In [13]:
def create_features_bow(single_image_descriptors, BoW, num_clusters):
    feature = np.array([0] * num_clusters)
    if single_image_descriptors is not None:
        distance = cdist(single_image_descriptors, BoW, metric='euclidean')
        argmin = np.argmin(distance, axis=1)
        for j in argmin:
            feature[j] += 1
    return feature

In [14]:
features = []


for i in image_features:
    feature = create_features_bow(i, BoW, num_clusters)
    features.append(feature)




225
